### Creating an image classifier able to find sneakers in instagram posts

The data comprises of few thousand images of sneakers collected using google images and instagram
and few thousand images of sneakers.    
Your goal is to use what you learned from previous examples and create a sneaker-not-sneaker binary classifier.

The task comprises of multiple sub-tasks that you need to do to build the classifier.

1. Create a dataset able to load data from new_meta_sneakers.csv
2. Create a fine tune binary classification architecture.
3. Create a training loop and train your model.

![title](sneakers.png)


### On the bottom of the following cell you see the data you will work with

In [3]:
%matplotlib inline
from torch import nn
import easyimages
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from pretrainedmodels.models import resnet50
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score

import matplotlib.pyplot as plt
import torch 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


In [2]:
DATASET_USAGE_PERCENTAGE = 0.1

base_path ='/media/i008/ssd500/fashion_classify_data/'
df = pd.read_csv(os.path.join(base_path,'new_meta_sneakers.csv')).sample(frac=1)
df.image_path = base_path +df.image_path
df.tags = df.tags.map({'sneakers': 1, 'negatives': 0})
df = df.sample(frac=DATASET_USAGE_PERCENTAGE)
df.head()

,Unnamed: 0,image_path,tags
16572,16572,/media/i008/ssd500/fashion_classify_data/sneak...,1
45581,45581,/media/i008/ssd500/fashion_classify_data/sneak...,1
27500,27500,/media/i008/ssd500/fashion_classify_data/sneak...,1
45655,45655,/media/i008/ssd500/fashion_classify_data/sneak...,1
34567,34567,/media/i008/ssd500/fashion_classify_data/negat...,0


### Creating the torch Dataset.

First thing we need to do is create a dataset able to load our data. Since our metadata is stored in a csv file, our 
dataset should accept this file as a base source of what needs to be loaded.

Our dataset should also support augumentations and a "inference" mode wich disables them for predicting.


In [168]:
import PIL
from PIL import Image
import io
import requests
import torch


class OneClassImageClassificationDataset(Dataset):
    def __init__(self, annotations, image_transform):
        """
        annotations is a pandas dataframe
        
        """
        super().__init__()
        self.annotations = annotations
        self.image_transform = image_transform

    def __len__(self):
        """
        Return the length of the annotations dataframe
        """
        # your code here
        return len(self.annotations)

    def __getitem__(self, index):
        """
        Using methods you wrote:
        1 - load image from disk for given index  (self.load_from_disk)
        2 - transform image (self.image_transform)
        3 - Load target (self.load_target)
        return Xi, yi
        """
        
        # YOUR CODE HERE

        return Xi, yi

    def load_to_pil(self, uri):
        """
        Write a helper function that uses PIL.Image to load a file and convert to RGB and returns it
        
        """
        image_pil =  # YOUR CODE HERE
        return image_pil


    def load_from_disk(self, index):
        """
        Loads an image from disk given a index.
        It gets the path of an image with the corresponding index from the metadata 
        It passes the URI to the self.load_to_pil and returns a PIL.Image
        """
        image_path = # YOUR CODE HERE
        return self.load_to_pil(image_path)

    def load_target(self, index):
        """
        This function should get the tag for a given index from the annotations dataframe
        You .iloc can become useful.    
        This methods should return, either a 0 or a 1.
        """
        
        #label = # YOUR CODE HERE

        return label
    
    
class BaseSampler(Sampler):
    def __init__(self, df, n_samples):
        self.df = df
        self.n_samples = n_samples
        
    def __iter__(self):
        return iter(self._get_sample())
        
    def __len__(self):
        return self.n_samples
    
    def _get_sample(self):
        return np.random.choice(len(self.df), self.n_samples, replace=False)
        

def binary_classification_model():
    """
    Write a function that loads a resnet50 model from pretrainedmodels. 
    - freezes its layers
    - replaces the last_linear with the proper output number. As we did in previous example.
    - replace avgpool with adaptiv pooling.
    """
    model = resnet50()

    # model = YOUR CODE HERE
    return model



In [175]:

# YOUR CODE HERE:
# SPLIT the dataframe into df_train, df_test (thing about using sklearn.model_selection.train_test_split)
df_train, df_test = train_test_split(df, train_size=0.8)
df_train = df_train.reset_index()
df_test = df_test.reset_index()

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
N_EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = 32

image_transform_train = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)])

# YOUR CODE define image_transform_test
image_transform_test = ?

# YOUR CODE define the crieterion
criterion = ? 


net = binary_classification_model()

optimizer = ?
# initialize the BaseSampler with 1000 samples per epoch
bs = ?

# YOUR CODE
# Instantiate the OneClassImageClassificationDatasets
train_ds = ?
test_ds = ?

#YOUR CODE
#Initialize your DataLoader (using datasets)
train_dl = ?
test_dl = ?

/home/i008/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
import numpy as np 
def evaluate_model(model, loader, print_info=False):
    with torch.no_grad():
        model.eval()
        collect_results = []
        collect_target = []
        for batch in loader:
            X, y = batch
            X = X.to(DEVICE)
            y = y.to(DEVICE).detach().cpu().numpy()
            pred = model(X)
            collect_results.append(pred.sigmoid().detach().cpu().numpy())
            collect_target.append(y) 
    
        preds_proba = np.concatenate(collect_results)
        preds = preds_proba.argmax(axis=1)
        
        targets = np.concatenate(collect_target)
        
        ll = log_loss(targets, preds_proba)
        acc = accuracy_score(targets, preds)
        if print_info:
            print("test log-loss: {}".format(ll))
            print("overall accuracy:  {}".format(acc))
            #print(classification_report(targets, preds))
        model.train()
        
        return ll, acc
        

# Training Loop

In [ ]:

metrics = []
metrics_names = ['loss_train','loss_test','acc_train','acc_test']
losses = []
net.to(DEVICE)

for epoch in range(N_EPOCHS):
    for X, y in train_dl:
        X = X.to(DEVICE)
        y = y.to(DEVICE)
        optimizer.zero_grad()
        ypred=net(X)
        loss = criterion(ypred, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
        
    testll, testacc = evaluate_model(net, test_dl)
    trainll, trainacc = evaluate_model(net, train_dl)
    print("test: {} {}".format(testll, testacc))
    print("train: {} {}".format(trainll, trainacc))
    metrics.append([trainll, testll, trainacc, testacc])

In [ ]:
%load sns.ipynb
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Creating an image classifier able to find sneakers in instagram posts\n",
    "\n",
    "The data comprises of few thousand images of sneakers collected using google images and instagram\n",
    "and few thousand images of sneakers.    \n",
    "Your goal is to use what you learned from previous examples and create a sneaker-not-sneaker binary classifier.\n",
    "\n",
    "The task comprises of multiple sub-tasks that you need to do to build the classifier.\n",
    "\n",
    "1. Create a dataset able to load data from new_meta_sneakers.csv\n",
    "2. Create a fine tune binary classification architecture.\n",
    "3. Create a training loop and train your model.\n",
    "\n",
    "![title](sneakers.png)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### On the bottom of the following cell you see the data you will work with"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 112,
   "metadata": {},
   "outputs": [],
   "source": [
    "%matplotlib inline\n",
    "from torch import nn\n",
    "import easyimages\n",
    "import pandas as pd\n",
    "import os\n",
    "from torch.utils.data import Dataset, DataLoader\n",
    "from pretrainedmodels.models import resnet50\n",
    "from torchvision.transforms import transforms\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import log_loss, accuracy_score\n",
    "\n",
    "import matplotlib.pyplot as plt\n",
    "import torch \n",
    "DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 113,
   "metadata": {
    "scrolled": true
   },
   "outputs": [],
   "source": [
    "DATASET_USAGE_PERCENTAGE = 0.1\n",
    "\n",
    "base_path ='/media/i008/ssd500/fashion_classify_data/'\n",
    "df = pd.read_csv(os.path.join(base_path,'new_meta_sneakers.csv')).sample(frac=1)\n",
    "df.image_path = base_path +df.image_path\n",
    "df.tags = df.tags.map({'sneakers': 1, 'negatives': 0})\n",
    "df = df.sample(frac=DATASET_USAGE_PERCENTAGE)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Creating the torch Dataset.\n",
    "\n",
    "First thing we need to do is create a dataset able to load our data. Since our metadata is stored in a csv file, our \n",
    "dataset should accept this file as a base source of what needs to be loaded.\n",
    "\n",
    "Our dataset should also support augumentations and a \"inference\" mode wich disables them for predicting.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 168,
   "metadata": {},
   "outputs": [],
   "source": [
    "import PIL\n",
    "from PIL import Image\n",
    "import io\n",
    "import requests\n",
    "import torch\n",
    "\n",
    "\n",
    "class OneClassImageClassificationDataset(Dataset):\n",
    "    def __init__(self, annotations, image_transform):\n",
    "        \"\"\"\n",
    "        annotations is a pandas dataframe\n",
    "        \n",
    "        \"\"\"\n",
    "        super().__init__()\n",
    "        self.annotations = annotations\n",
    "        self.image_transform = image_transform\n",
    "\n",
    "    def __len__(self):\n",
    "        \"\"\"\n",
    "        Return the length of the annotations dataframe\n",
    "        \"\"\"\n",
    "        # your code here\n",
    "        return len(self.annotations)\n",
    "\n",
    "    def __getitem__(self, index):\n",
    "        \"\"\"\n",
    "        Using methods you wrote:\n",
    "        1 - load image from disk for given index  (self.load_from_disk)\n",
    "        2 - transform image (self.image_transform)\n",
    "        3 - Load target (self.load_target)\n",
    "        return Xi, yi\n",
    "        \"\"\"\n",
    "        \n",
    "        # YOUR CODE HERE\n",
    "\n",
    "        Xi = self.load_from_disk(index)\n",
    "        Xi = self.image_transform(Xi)\n",
    "        yi = self.load_target(index)\n",
    "        return Xi, yi\n",
    "\n",
    "    def load_to_pil(self, uri):\n",
    "        \"\"\"\n",
    "        Write a helper function that uses PIL.Image to load a file and returns it\n",
    "        \"\"\"\n",
    "\n",
    "        image_pil = Image.open(uri)\n",
    "        image_pil = image_pil.convert(\"RGB\")\n",
    "        # image_pil = YOUR CODE HERE\n",
    "        return image_pil\n",
    "\n",
    "\n",
    "    def load_from_disk(self, index):\n",
    "        \"\"\"\n",
    "        Loads an image from disk given a index.\n",
    "        It gets the path of an image with the corresponding index from the metadata \n",
    "        It passes the URI to the self.load_to_pil and returns a PIL.Image\n",
    "        \"\"\"\n",
    "        image_path = self.annotations.iloc[index]['image_path']\n",
    "        #image_path = # YOUR CODE HERE\n",
    "        return self.load_to_pil(image_path)\n",
    "\n",
    "    def load_target(self, index):\n",
    "        \"\"\"\n",
    "        This function should get the tag for a given index from the annotations dataframe\n",
    "        You .iloc can become useful.    \n",
    "        This methods should return, either a 0 or a 1.\n",
    "        \"\"\"\n",
    "        \n",
    "        #label = # YOUR CODE HERE\n",
    "        label = self.annotations.iloc[index]['tags']\n",
    "\n",
    "        return label\n",
    "    \n",
    "    \n",
    "class BaseSampler(Sampler):\n",
    "    def __init__(self, df, n_samples):\n",
    "        self.df = df\n",
    "        self.n_samples = n_samples\n",
    "        \n",
    "    def __iter__(self):\n",
    "        return iter(self._get_sample())\n",
    "        \n",
    "    def __len__(self):\n",
    "        return self.n_samples\n",
    "    \n",
    "    def _get_sample(self):\n",
    "        return np.random.choice(len(self.df), self.n_samples, replace=False)\n",
    "        \n",
    "\n",
    "def binary_classification_model():\n",
    "    \"\"\"\n",
    "    Write a function that loads a resnet50 model from pretrainedmodels, freezes its layers\n",
    "    replaces the last_linear with the proper output number. As we did in previous example.\n",
    "    replace avgpool with adaptiv pooling.\n",
    "    \"\"\"\n",
    "    model = resnet50()\n",
    "    for p in model.parameters():\n",
    "        p.requires_grad = False\n",
    "    inft = model.last_linear.in_features\n",
    "    model.last_linear = nn.Linear(in_features=inft, out_features=2)\n",
    "    model.avgpool = nn.AdaptiveAvgPool2d(1)\n",
    "    \n",
    "    # model = YOUR CODE HERE\n",
    "    return model\n",
    "\n",
    "\n",
    "from imgaug import augmenters as iaa\n",
    "\n",
    "aug_seq = iaa.Sequential([\n",
    "    iaa.Fliplr(p=0.5),\n",
    "    iaa.Sometimes(\n",
    "        0.3,\n",
    "        iaa.Multiply((0.9, 1.2))\n",
    "    ),\n",
    "    iaa.Sometimes(\n",
    "        0.3,\n",
    "        iaa.AdditiveGaussianNoise()\n",
    "    ),\n",
    "    iaa.Affine(\n",
    "        scale=(0.5, 2),\n",
    "        translate_percent=(-0.2, 0.2)\n",
    "    )\n",
    "])\n",
    "def augment(self, augmenter, image):\n",
    "    augmenter = augmenter.to_deterministic()\n",
    "    img_aug = augmenter.augment_image(np.array(image))\n",
    "    img_aug = Image.fromarray(img_aug)\n",
    "    return img_aug"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 175,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/home/i008/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.\n",
      "  FutureWarning)\n"
     ]
    }
   ],
   "source": [
    "\n",
    "# YOUR CODE HERE:\n",
    "# SPLIT the dataframe into df_train, df_test (thing about using sklearn.model_selection.train_test_split)\n",
    "df_train, df_test = train_test_split(df, train_size=0.8)\n",
    "df_train = df_train.reset_index()\n",
    "df_test = df_test.reset_index()\n",
    "\n",
    "MEAN = [0.485, 0.456, 0.406]\n",
    "STD = [0.229, 0.224, 0.225]\n",
    "N_EPOCHS = 10\n",
    "BATCH_SIZE = 32\n",
    "IMAGE_SIZE = 32\n",
    "\n",
    "image_transform_train = transforms.Compose([\n",
    "    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),\n",
    "    transforms.ToTensor(),\n",
    "    transforms.Normalize(mean=MEAN, std=STD)])\n",
    "\n",
    "# YOUR CODE define image_transform_test\n",
    "image_transform_test = transforms.Compose([\n",
    "    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),\n",
    "    transforms.ToTensor(),\n",
    "    transforms.Normalize(mean=MEAN, std=STD)])\n",
    "\n",
    "# YOUR CODE define the crieterion\n",
    "criterion = nn.CrossEntropyLoss()\n",
    "\n",
    "net = binary_classification_model()\n",
    "\n",
    "optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)\n",
    "\n",
    "# initialize the BaseSampler\n",
    "bs = BaseSampler(train_ds, 1000)\n",
    "\n",
    "# YOUR CODE\n",
    "# Instantiate the OneClassImageClassificationDatasets\n",
    "train_ds = OneClassImageClassificationDataset(df_train, image_transform=image_transform_train)\n",
    "test_ds = OneClassImageClassificationDataset(df_test, image_transform=image_transform_test)\n",
    "\n",
    "#YOUR CODE\n",
    "#Initialize your DataLoader (using datasets)\n",
    "train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=bs)\n",
    "test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 176,
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "test: 0.6851049539626844 0.5550335570469799\n",
      "train: 0.6767495510429143 0.579\n",
      "test: 0.6824289217291263 0.5718120805369128\n",
      "train: 0.6819015721082687 0.581\n",
      "test: 0.6811918862101516 0.6161073825503356\n",
      "train: 0.6855373743530363 0.62\n",
      "test: 0.6765110107951317 0.6073825503355704\n",
      "train: 0.6636039858073928 0.621\n",
      "test: 0.6719322499876917 0.6241610738255033\n",
      "train: 0.6605288547966629 0.634\n"
     ]
    },
    {
     "ename": "KeyboardInterrupt",
     "evalue": "",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mKeyboardInterrupt\u001b[0m                         Traceback (most recent call last)",
      "\u001b[0;32m<ipython-input-176-f8e01ef63f78>\u001b[0m in \u001b[0;36m<module>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     46\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     47\u001b[0m     \u001b[0mtestll\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtestacc\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mevaluate_model\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mnet\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtest_dl\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 48\u001b[0;31m     \u001b[0mtrainll\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtrainacc\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mevaluate_model\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mnet\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtrain_dl\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     49\u001b[0m     \u001b[0mprint\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"test: {} {}\"\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mformat\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mtestll\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtestacc\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     50\u001b[0m     \u001b[0mprint\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"train: {} {}\"\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mformat\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mtrainll\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtrainacc\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m<ipython-input-176-f8e01ef63f78>\u001b[0m in \u001b[0;36mevaluate_model\u001b[0;34m(model, loader, print_info)\u001b[0m\n\u001b[1;32m      5\u001b[0m         \u001b[0mcollect_results\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      6\u001b[0m         \u001b[0mcollect_target\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m----> 7\u001b[0;31m         \u001b[0;32mfor\u001b[0m \u001b[0mbatch\u001b[0m \u001b[0;32min\u001b[0m \u001b[0mloader\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m      8\u001b[0m             \u001b[0mX\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0my\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mbatch\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      9\u001b[0m             \u001b[0mX\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mX\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mto\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mDEVICE\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py\u001b[0m in \u001b[0;36m__next__\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    613\u001b[0m         \u001b[0;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mnum_workers\u001b[0m \u001b[0;34m==\u001b[0m \u001b[0;36m0\u001b[0m\u001b[0;34m:\u001b[0m  \u001b[0;31m# same-process loading\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    614\u001b[0m             \u001b[0mindices\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mnext\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0msample_iter\u001b[0m\u001b[0;34m)\u001b[0m  \u001b[0;31m# may raise StopIteration\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 615\u001b[0;31m             \u001b[0mbatch\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcollate_fn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdataset\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mi\u001b[0m\u001b[0;34m]\u001b[0m \u001b[0;32mfor\u001b[0m \u001b[0mi\u001b[0m \u001b[0;32min\u001b[0m \u001b[0mindices\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    616\u001b[0m             \u001b[0;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpin_memory\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    617\u001b[0m                 \u001b[0mbatch\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mpin_memory_batch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mbatch\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py\u001b[0m in \u001b[0;36m<listcomp>\u001b[0;34m(.0)\u001b[0m\n\u001b[1;32m    613\u001b[0m         \u001b[0;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mnum_workers\u001b[0m \u001b[0;34m==\u001b[0m \u001b[0;36m0\u001b[0m\u001b[0;34m:\u001b[0m  \u001b[0;31m# same-process loading\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    614\u001b[0m             \u001b[0mindices\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mnext\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0msample_iter\u001b[0m\u001b[0;34m)\u001b[0m  \u001b[0;31m# may raise StopIteration\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 615\u001b[0;31m             \u001b[0mbatch\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcollate_fn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdataset\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mi\u001b[0m\u001b[0;34m]\u001b[0m \u001b[0;32mfor\u001b[0m \u001b[0mi\u001b[0m \u001b[0;32min\u001b[0m \u001b[0mindices\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    616\u001b[0m             \u001b[0;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpin_memory\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    617\u001b[0m                 \u001b[0mbatch\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mpin_memory_batch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mbatch\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m<ipython-input-168-715f597598a7>\u001b[0m in \u001b[0;36m__getitem__\u001b[0;34m(self, index)\u001b[0m\n\u001b[1;32m     34\u001b[0m         \u001b[0;31m# YOUR CODE HERE\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     35\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 36\u001b[0;31m         \u001b[0mXi\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mload_from_disk\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mindex\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     37\u001b[0m         \u001b[0mXi\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mimage_transform\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mXi\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     38\u001b[0m         \u001b[0myi\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mload_target\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mindex\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m<ipython-input-168-715f597598a7>\u001b[0m in \u001b[0;36mload_from_disk\u001b[0;34m(self, index)\u001b[0m\n\u001b[1;32m     58\u001b[0m         \u001b[0mimage_path\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mannotations\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0miloc\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mindex\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0;34m'image_path'\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     59\u001b[0m         \u001b[0;31m#image_path = # YOUR CODE HERE\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 60\u001b[0;31m         \u001b[0;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mload_to_pil\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mimage_path\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     61\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     62\u001b[0m     \u001b[0;32mdef\u001b[0m \u001b[0mload_target\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mself\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mindex\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m<ipython-input-168-715f597598a7>\u001b[0m in \u001b[0;36mload_to_pil\u001b[0;34m(self, uri)\u001b[0m\n\u001b[1;32m     45\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     46\u001b[0m         \u001b[0mimage_pil\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mImage\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mopen\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0muri\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 47\u001b[0;31m         \u001b[0mimage_pil\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mimage_pil\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mconvert\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"RGB\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     48\u001b[0m         \u001b[0;31m# image_pil = YOUR CODE HERE\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     49\u001b[0m         \u001b[0;32mreturn\u001b[0m \u001b[0mimage_pil\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/lib/python3.6/site-packages/PIL/Image.py\u001b[0m in \u001b[0;36mconvert\u001b[0;34m(self, mode, matrix, dither, palette, colors)\u001b[0m\n\u001b[1;32m    890\u001b[0m         \"\"\"\n\u001b[1;32m    891\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 892\u001b[0;31m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mload\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    893\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    894\u001b[0m         \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0mmode\u001b[0m \u001b[0;32mand\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmode\u001b[0m \u001b[0;34m==\u001b[0m \u001b[0;34m\"P\"\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/anaconda3/lib/python3.6/site-packages/PIL/ImageFile.py\u001b[0m in \u001b[0;36mload\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    233\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    234\u001b[0m                             \u001b[0mb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mb\u001b[0m \u001b[0;34m+\u001b[0m \u001b[0ms\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 235\u001b[0;31m                             \u001b[0mn\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0merr_code\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mdecoder\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdecode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mb\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    236\u001b[0m                             \u001b[0;32mif\u001b[0m \u001b[0mn\u001b[0m \u001b[0;34m<\u001b[0m \u001b[0;36m0\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    237\u001b[0m                                 \u001b[0;32mbreak\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;31mKeyboardInterrupt\u001b[0m: "
     ]
    }
   ],
   "source": [
    "import numpy as np \n",
    "def evaluate_model(model, loader, print_info=False):\n",
    "    with torch.no_grad():\n",
    "        model.eval()\n",
    "        collect_results = []\n",
    "        collect_target = []\n",
    "        for batch in loader:\n",
    "            X, y = batch\n",
    "            X = X.to(DEVICE)\n",
    "            y = y.to(DEVICE).detach().cpu().numpy()\n",
    "            pred = model(X)\n",
    "            collect_results.append(pred.sigmoid().detach().cpu().numpy())\n",
    "            collect_target.append(y) \n",
    "    \n",
    "        preds_proba = np.concatenate(collect_results)\n",
    "        preds = preds_proba.argmax(axis=1)\n",
    "        \n",
    "        targets = np.concatenate(collect_target)\n",
    "        \n",
    "        ll = log_loss(targets, preds_proba)\n",
    "        acc = accuracy_score(targets, preds)\n",
    "        if print_info:\n",
    "            print(\"test log-loss: {}\".format(ll))\n",
    "            print(\"overall accuracy:  {}\".format(acc))\n",
    "            #print(classification_report(targets, preds))\n",
    "        model.train()\n",
    "        \n",
    "        return ll, acc\n",
    "    \n",
    "metrics = []\n",
    "metrics_names = ['loss_train','loss_test','acc_train','acc_test']\n",
    "losses = []\n",
    "\n",
    "net.to(DEVICE)\n",
    "\n",
    "for epoch in range(N_EPOCHS):\n",
    "    for X, y in train_dl:\n",
    "        X = X.to(DEVICE)\n",
    "        y = y.to(DEVICE)\n",
    "        optimizer.zero_grad()\n",
    "        ypred=net(X)\n",
    "        loss = criterion(ypred, y)\n",
    "        loss.backward()\n",
    "        optimizer.step()\n",
    "        losses.append(loss.detach().cpu().numpy())\n",
    "        \n",
    "    testll, testacc = evaluate_model(net, test_dl)\n",
    "    trainll, trainacc = evaluate_model(net, train_dl)\n",
    "    print(\"test: {} {}\".format(testll, testacc))\n",
    "    print(\"train: {} {}\".format(trainll, trainacc))\n",
    "    metrics.append([trainll, testll, trainacc, testacc])\n",
    "        "
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [default]",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
